# definitions

In [954]:
import json
import sys
sys.path.append('../py')
import myutils as my
import importlib
importlib.reload(my)
import json
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Rectangle
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
from matplotlib.ticker import MaxNLocator
from matplotlib import rc
from matplotlib.gridspec import GridSpec
from matplotlib import ticker

# plt.rcParams["font.family"] = "serif"
# plt.rcParams["mathtext.fontset"] = "dejavuserif"

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
matplotlib.rcParams['agg.path.chunksize'] = 10000

recalculate = recompile = True

In [ ]:
def set_ticks(ax_all, N=6, cut_x = 0.15, cut_y = 0.2):
    for ax in ax_all:
        ax.tick_params(axis='both', labelsize=6)
        ax.margins(x=0.)

        box = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        box_w = box.width
        box_h = box.height

        if ax.get_xscale() == "log":
            ax.xaxis.set_major_locator(ticker.LogLocator(numticks=N))
        else:
            ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=N))

        if ax.get_yscale() == "log":
            ax.yaxis.set_major_locator(ticker.LogLocator(numticks=N))
            ax.yaxis.set_minor_locator(ticker.LogLocator(subs="auto", numticks=10))
        else:
            ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=N))

        (x0, x1) = ax.get_xlim()
        (y0, y1) = ax.get_ylim()


        x0 = ax.dataLim.x0
        x1 = ax.dataLim.x1
        y0 = ax.dataLim.y0
        y1 = ax.dataLim.y1



        axis_to_data = ax.transAxes + ax.transData.inverted()
        (x2, y2) = axis_to_data.transform([1 - cut_x/box.width, 1 - cut_y/box.height])
        (x0, y0) = axis_to_data.transform([-0.1, -0.1])

        # print(x0, x1, x2)
        # print(y0, y1, y2)


        ax.set_xticks([tick for tick in ax.get_xticks() if tick <= x2])
        ax.set_yticks([tick for tick in ax.get_yticks() if tick >= y0 and tick <= y2])

        # print(ax.get_yticklabels())

        if ax.get_yscale() == "log":       
            ax.set_yticklabels([label.get_text().replace("-", "\\mbox{-}") for label in ax.get_yticklabels()])
        # print(ax.get_yticklabels())

def set_labels(ax, x_label, y_label):
    box = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    
    if x_label:
        ax.text(1, -0.05/box.height, x_label, transform=ax.transAxes, va="top", ha="right")
    if y_label:
        ax.text(-0.03/box.width, 1,  y_label, transform=ax.transAxes, va="top", ha="right")

# Equations

## Relay1

In [ ]:
!pwd

In [ ]:
recompile = True
recalculate = True

script = "ddes_test"
params = dict(
    eq = 'Relay1',
     c = 0,
    d0 = 1,
    d1 = -0.5,
    tau = 1,
    h = 0.13,
    t_finish = 10
)

params_str = json.dumps(params)

output_filename = script + " " + params_str
output_filename = output_filename[:min(len(output_filename), 200)] # truncate by 200 characters


if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params_str=params_str, output_filename=output_filename, recompile=recompile, flags = "")

filename_bin = f"../output/bin/{output_filename}.bin"
output = my.get_binary(filename_bin)
t = output.T[0]
x = output.T[1]
# le = output.T[1]
# x = output[:, 1:].T
plt.plot(t, x)
# plt.scatter(t, x)
print(t, x)

In [ ]:
fig = plt.figure(figsize=(8,2),dpi=1000)

gs = GridSpec(nrows=2, ncols=2, width_ratios=[2, 2], height_ratios=[3,1],hspace=0.1)

ax_sol  = plt.subplot(gs[:,0])
ax_le   = plt.subplot(gs[0,1])
ax_le_std = plt.subplot(gs[1,1], sharex = ax_le)


ax_sol.plot(t,x,'k',lw=0.5)
ax_sol.plot(t+1,x,'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 20)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)

ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2)/2;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.4g}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_le, ax_le_std]
for ax in ax_all:
    ax.tick_params(axis='both', labelsize=6)
    ax.margins(x=0.)
    
    box = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    box_w = box.width
    box_h = box.height
    
    N = 5
    if ax.get_xscale() == "log":
        ax.xaxis.set_major_locator(ticker.LogLocator(numticks=N))
    else:
        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=N))
    
    if ax.get_yscale() == "log":
        ax.yaxis.set_major_locator(ticker.LogLocator(numticks=N))
    else:
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=N))
    
    (x0, x1) = ax.get_xlim()
    (y0, y1) = ax.get_ylim()
    
    
    x0 = ax.dataLim.x0
    x1 = ax.dataLim.x1
    y0 = ax.dataLim.y0
    y1 = ax.dataLim.y1
    

    print(box.width, box.height)
    
    axis_to_data = ax.transAxes + ax.transData.inverted()
    (x2, y2) = axis_to_data.transform([1 - 0.15/box.width, 1 - 0.15/box.height])
    (x0, y0) = axis_to_data.transform([-0.1, -0.1])
    
    print()
    # print(x0, x1, x2)
    # print(y0, y1, y2)
    
   
    ax.set_xticks([tick for tick in ax.get_xticks() if tick <= x2])
    ax.set_yticks([tick for tick in ax.get_yticks() if tick >= y0 and tick <= y2])
    
    # print(ax.get_yticklabels())

    if ax.get_yscale() == "log":       
        ax.set_yticklabels([label.get_text().replace("-", "\\mbox{-}") for label in ax.get_yticklabels()])
    # print(ax.get_yticklabels())
    
ax_sol.set_xlabel("$t$",loc="right",labelpad=0, va="bottom")
ax_sol.set_ylabel("$x(t)$",rotation=0,loc="top",labelpad=-20, va='top')
ax_le_std.set_xlabel("$t$",loc="right",labelpad=0, va="bottom")
ax_le.set_ylabel("$\lambda(t)$", rotation=0, loc="top", labelpad=-20, va='top')
ax_le_std.set_ylabel("$\sigma(t)$", rotation=0, loc="top", labelpad=-15, va='center')

plt.savefig(f"../output/{filename}.jpg", format="jpg", dpi=1000,bbox_inches='tight')


## Relay2

### Lorenz picture

In [ ]:
recompile = True
recalculate = True

script = "ddes_test"
params = dict(
    eq = 'Relay2', b = -0.2, c = 1, d = 1, tau = 1,
    x0=0, v0 = 1, 
    h=0.01, t_finish = 50000
)
params_str = json.dumps(params)

output_filename = script + " " + params_str
output_filename = output_filename[:min(len(output_filename), 200)] # truncate by 200 characters


if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params_str=params_str, output_filename=output_filename, recompile=recompile, flags = "-Wextra -Werror -O3")

filename_bin = f"../output/bin/{output_filename}.bin"
output = my.get_binary(filename_bin)
t = output.T[0]
# le = output.T[1]
x = output[:, 1:].T

# le = output.T[1]

# plt.plot(t, x[0])

In [ ]:
fig = plt.figure(figsize=(5,1.4),dpi=1000)

# gs = GridSpec(nrows=2, ncols=3, width_ratios=[1, 1, 1.3], height_ratios=[3,2],hspace=0.15,wspace=0.25)
gs = GridSpec(nrows=2, ncols=2, width_ratios=[2, 1,], height_ratios=[3,2],hspace=0.15,wspace=0.2)

ax_sol  = plt.subplot(gs[:,0])
ax_sol2  = plt.subplot(gs[:,1])
# ax_le   = plt.subplot(gs[0,2])
# ax_le_std = plt.subplot(gs[1,2], sharex = ax_le)


ax_sol.plot(t,x[0],'k',lw=0.5)
ax_sol.plot(t+params["tau"],x[0],'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 200)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
# ax_le.plot(t, le, 'k', lw=0.5)
# ax_le.tick_params(labelbottom=False)

# step = int(10*len(t)/params["t_finish"])
# for i in range(0, len(t), step):
#     ax_sol2.plot(x[0][i:i+step], x[1][i:i+step], 'k', lw=0.05, alpha=0.01)
    
ax_sol2.plot(x[0], x[1],'k',lw=0.005, rasterized=True)
# ax_sol2.plot(x[0], x[1],'k',lw=0.5, alpha=0.01)
# ax_sol.plot(t+1,x[0],'k',lw=0.3, linestyle=":")
# ax_sol.set_xlim(0, 200)
# # ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
# ax_le.plot(t, le, 'k', lw=0.5)
# ax_le.tick_params(labelbottom=False)


ax_sol2.axhline(0,linestyle=':',c='k',lw=0.3)
ax_sol2.axvline(0,linestyle=':',c='k',lw=0.3)

# le_mean = np.mean(le)
# le_delta1 = np.max(le - le_mean);
# le_delta2 = np.max(le_mean - le);
# le_delta = min(le_delta1, le_delta2)/4;
# ax_le.dataLim.y0 = le_mean - le_delta;
# ax_le.dataLim.y1 = le_mean + le_delta;

# window = len(le)//10
# le_std = pd.Series(le).rolling(window=window).std()
# ax_le_std.plot(t, le_std, 'k', lw=0.5)
# ax_le_std.set_yscale("log")
# le_final = np.mean(le[-window:])


# ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.4g}$',va="bottom", ha="right")

# ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_sol2, ax_le, ax_le_std]
set_ticks(ax_all, N=4, cut_x = 0.3)

ax_sol2.margins(x=0.05,y=0.05)


set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_sol2, "$x(t)$", "$\dot x(t)$")
# set_labels(ax_le, None, "$\lambda(t)$")
# set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"../output/{output_filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01,transparent=True)

In [ ]:
fig = plt.figure(figsize=(8,1.25),dpi=1000)

# gs = GridSpec(nrows=2, ncols=3, width_ratios=[1, 1, 1.3], height_ratios=[3,2],hspace=0.15,wspace=0.25)
gs = GridSpec(nrows=2, ncols=3, width_ratios=[2, 1, 2], height_ratios=[3,2],hspace=0.15,wspace=0.2)

ax_sol  = plt.subplot(gs[:,0])
ax_sol2  = plt.subplot(gs[:,1])
ax_le   = plt.subplot(gs[0,2])
ax_le_std = plt.subplot(gs[1,2], sharex = ax_le)


ax_sol.plot(t,x[0],'k',lw=0.5)
ax_sol.plot(t+params["tau"],x[0],'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 200)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

# step = int(10*len(t)/params["t_finish"])
# for i in range(0, len(t), step):
#     ax_sol2.plot(x[0][i:i+step], x[1][i:i+step], 'k', lw=0.05, alpha=0.01)
    
ax_sol2.plot(x[0], x[1],'k',lw=0.005, rasterized=True)
# ax_sol2.plot(x[0], x[1],'k',lw=0.5, alpha=0.01)
# ax_sol.plot(t+1,x[0],'k',lw=0.3, linestyle=":")
# ax_sol.set_xlim(0, 200)
# # ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
# ax_le.plot(t, le, 'k', lw=0.5)
# ax_le.tick_params(labelbottom=False)


ax_sol2.axhline(0,linestyle=':',c='k',lw=0.3)
ax_sol2.axvline(0,linestyle=':',c='k',lw=0.3)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2)/4;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.4g}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_sol2, ax_le, ax_le_std]
set_ticks(ax_all, N=4, cut_x = 0.3)

ax_sol2.margins(x=0.05,y=0.05)


set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_sol2, "$x(t)$", "$\dot x(t)$")
set_labels(ax_le, None, "$\lambda(t)$")
set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"output_img/{filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01,transparent=True)


### Cycle

In [ ]:
recompile = False
params = dict(
    eq = 'Relay2',
     b = -0.2,
     c = 3, d = 3, x0=0., v0 = 1.5, tau = 1,
     h=0.01, t_finish = 5000
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)

# fig, ax = plt.subplots(1,3, figsize=(8,7/3),dpi=1000)
output = my.get_binary(filename_bin)
t = output.T[0]
le = output.T[1]
x = output[:, 2:].T

In [ ]:
fig = plt.figure(figsize=(8,1.25),dpi=1000)

# gs = GridSpec(nrows=2, ncols=3, width_ratios=[1, 1, 1.3], height_ratios=[3,2],hspace=0.15,wspace=0.25)
gs = GridSpec(nrows=2, ncols=3, width_ratios=[2, 1, 2], height_ratios=[3,2],hspace=0.15,wspace=0.25)

ax_sol  = plt.subplot(gs[:,0])
ax_sol2  = plt.subplot(gs[:,1])
ax_le   = plt.subplot(gs[0,2])
ax_le_std = plt.subplot(gs[1,2], sharex = ax_le)


ax_sol.plot(t,x[0],'k',lw=0.5)
ax_sol.plot(t+params["tau"],x[0],'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 50)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

ax_sol2.plot(x[0], x[1],'k',lw=0.2)
ax_sol2.axhline(0,linestyle=':',c='k',lw=0.3)
ax_sol2.axvline(0,linestyle=':',c='k',lw=0.3)

# ax_sol.plot(t+1,x[0],'k',lw=0.3, linestyle=":")
# ax_sol.set_xlim(0, 200)
# # ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
# ax_le.plot(t, le, 'k', lw=0.5)
# ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2)/4;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.4g}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_sol2, ax_le, ax_le_std]
set_ticks(ax_all, N=4, cut_x = 0.3)

ax_sol2.margins(x=0.05,y=0.05)


set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_sol2, "$x(t)$", "$\dot x(t)$")
set_labels(ax_le, None, "$\lambda(t)$")
set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"output_img/{filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01)


### Rosler picture

In [ ]:
params = dict(
    eq = 'Relay2',
     b = -0.02,
     c = 1, d = 1, x0=0, v0 = 2, tau = 1,
     h=0.01, t_finish = 100
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)

# fig, ax = plt.subplots(1,3, figsize=(8,7/3),dpi=1000)
output = my.get_binary(filename_bin)
t = output.T[0]
le = output.T[1]
x = output[:, 2:].T

In [ ]:
fig = plt.figure(figsize=(8,2),dpi=1000)

gs = GridSpec(nrows=2, ncols=3, width_ratios=[1, 1, 1.3], height_ratios=[3,1],hspace=0.1)

ax_sol  = plt.subplot(gs[:,0])
ax_sol2  = plt.subplot(gs[:,1])
ax_le   = plt.subplot(gs[0,2])
ax_le_std = plt.subplot(gs[1,2], sharex = ax_le)


ax_sol.plot(t,x[0],'k',lw=0.5)
ax_sol.plot(t+1,x[0],'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 500)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

ax_sol2.plot(x[0], x[1],'k',lw=0.001)
# ax_sol.plot(t+1,x[0],'k',lw=0.3, linestyle=":")
# ax_sol.set_xlim(0, 200)
# # ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
# ax_le.plot(t, le, 'k', lw=0.5)
# ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2)/4;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.4g}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_sol2, ax_le, ax_le_std]

for ax in ax_all:
    ax.tick_params(axis='both', labelsize=6)
    ax.margins(x=0.)
    
    N = 5
    if ax.get_xscale() == "log":
        ax.xaxis.set_major_locator(ticker.LogLocator(numticks=N))
    else:
        ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=N))
    
    if ax.get_yscale() == "log":
        ax.yaxis.set_major_locator(ticker.LogLocator(numticks=N))
    else:
        ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=N))
    
    (x0, x1) = ax.get_xlim()
    (y0, y1) = ax.get_ylim()
    
    
    x0 = ax.dataLim.x0
    x1 = ax.dataLim.x1
    y0 = ax.dataLim.y0
    y1 = ax.dataLim.y1
    

    
    axis_to_data = ax.transAxes + ax.transData.inverted()
    (x2, y2) = axis_to_data.transform([0.9, 0.9])
    (x0, y0) = axis_to_data.transform([-0.1, -0.1])
    
    print()
    print(x0, x1, x2)
    print(y0, y1, y2)
    
   
    ax.set_xticks([tick for tick in ax.get_xticks() if tick <= x2])
    ax.set_yticks([tick for tick in ax.get_yticks() if tick >= y0 and tick <= y2])
    
    # print(ax.get_yticklabels())

    if ax.get_yscale() == "log":       
        ax.set_yticklabels([label.get_text().replace("-", "\\mbox{-}") for label in ax.get_yticklabels()])
    # print(ax.get_yticklabels())
    
ax_sol2.margins(x=0.1,y=0.1)
    
    
ax_sol.set_xlabel("$t$",loc="right",labelpad=0, va="bottom")
ax_sol.set_ylabel("$x(t)$",rotation=0,loc="top",labelpad=-20, va='top')

ax_sol2.set_xlabel("$x(t)$",loc="right",labelpad=0, va="bottom")
ax_sol2.set_ylabel("$\dot x(t)$",rotation=0,loc="top",labelpad=-20, va='top')

ax_le.set_ylabel("$\lambda(t)$", rotation=0, loc="top", labelpad=-20, va='top')
ax_le_std.set_xlabel("$t$",loc="right",labelpad=0, va="bottom")
ax_le_std.set_ylabel("$\sigma(t)$", rotation=0, loc="top", labelpad=-17, va='top')




## Random walk equation

### Random case

In [ ]:
recompile = False

params = dict(
    eq = 'RandomWalk',
     b = 0.1,
    d0 = -1,
    d1 =  1,
    tau = 20,
    
    x0 = 5,
    
    h = 0.01,
    t_finish = 100000
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)

# fig, ax = plt.subplots(1,3, figsize=(8,7/3),dpi=1000)
t, le, x = my.get_binary(filename_bin).T

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
le_final = np.mean(le[-window:])


In [ ]:
fig = plt.figure(figsize=(8,1.25),dpi=1000)


gs = GridSpec(nrows=2, ncols=2, width_ratios=[2, 2], height_ratios=[3,2],hspace=0.15,wspace=0.15)

ax_sol  = plt.subplot(gs[:,0])
ax_le   = plt.subplot(gs[0,1])
ax_le_std = plt.subplot(gs[1,1], sharex = ax_le)


ax_sol.plot(t,x,'k',lw=0.5)
ax_sol.plot(t+params["tau"],x,'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 500)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)

ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = max(le_delta1, le_delta2)/8;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.4g}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_le, ax_le_std]
set_ticks(ax_all)

set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_le, None, "$\lambda(t)$")
set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"output_img/{filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01)



### Cycle case

In [ ]:
recompile = False

params = dict(
    eq = 'RandomWalk',
     b = 0.1,
    d0 = -1,
    d1 =  1,
    tau = 10,
    
    x0 = 5,
    
    h = 0.01,
    t_finish = 50000
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)

# fig, ax = plt.subplots(1,3, figsize=(8,7/3),dpi=1000)
t, le, x = my.get_binary(filename_bin).T

In [ ]:
fig = plt.figure(figsize=(8,1.25),dpi=1000)

gs = GridSpec(nrows=2, ncols=2, width_ratios=[2, 2], height_ratios=[3,2],hspace=0.15,wspace=0.2)

ax_sol  = plt.subplot(gs[:,0])
ax_le   = plt.subplot(gs[0,1])
ax_le_std = plt.subplot(gs[1,1], sharex = ax_le)



ax_sol.plot(t,x,'k',lw=0.5)
ax_sol.plot(t+params["tau"],x,'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 500)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)

ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2);
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.8f}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_le, ax_le_std]
set_ticks(ax_all)

set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_le, None, "$\lambda(t)$")
set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"output_img/{filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01)




## Makey Glass

In [ ]:
recompile = True

params = dict(
    eq = 'MakeyGlass',
     b = 0.9,
    d0 = 2,
    d1 =  0,
    tau = 10,
    x0 = 0.2,
    v0 = 2,
    h = 0.01,
    t_finish = 10000
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)

# fig, ax = plt.subplots(1,3, figsize=(8,7/3),dpi=1000)
t, le, x = my.get_binary(filename_bin).T

In [ ]:
fig = plt.figure(figsize=(8,1.25),dpi=1000)


gs = GridSpec(nrows=2, ncols=2, width_ratios=[3, 2], height_ratios=[3,2],hspace=0.15,wspace=0.2)

ax_sol  = plt.subplot(gs[:,0])
# ax_sol2 = plt.subplot(gs[1,0], sharex = ax_sol)
ax_le   = plt.subplot(gs[0,1])
ax_le_std = plt.subplot(gs[1,1], sharex = ax_le)


ax_sol.plot(t,x,'k',lw=0.5)
ax_sol.plot(t+params["tau"],x,'k',lw=0.3, linestyle=":")
start = 0
ax_sol.set_xlim(start, start+ 100)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
ax_sol.axhline(1,linestyle=':',c='k',lw=0.3)

ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2)/4;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])


ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.8f}$',va="bottom", ha="right")

ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)


ax_all = [ax_sol, ax_le, ax_le_std]

set_ticks(ax_all)

set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_le, None, "$\lambda(t)$")
set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"output_img/{filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01)


In [ ]:
recompile = False

params = dict(
    eq = 'MakeyGlass',
     b = 0.5,
    d0 = 2,
    d1 =  0,
    tau = 10,
    x0 = 0.2,
    v0 = 2,
    h = 0.001,
    t_finish = 5000
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)

# fig, ax = plt.subplots(1,3, figsize=(8,7/3),dpi=1000)
t, le, x = my.get_binary(filename_bin).T

# x = np.exp(y)
# y = np.log(x)


In [ ]:
fig = plt.figure(figsize=(8,1.25),dpi=1000)

gs = GridSpec(nrows=2, ncols=2, width_ratios=[3, 2], height_ratios=[3,2],hspace=0.15,wspace=0.2)

ax_sol  = plt.subplot(gs[:,0])
# ax_sol2 = plt.subplot(gs[1,0], sharex = ax_sol)
ax_le   = plt.subplot(gs[0,1])
ax_le_std = plt.subplot(gs[1,1], sharex = ax_le)



# ax_sol2.plot(t,y,'k',lw=0.5)
# ax_sol2.plot(t+1,y,'k',lw=0.3, linestyle=":")
# ax_sol2.axhline(0,linestyle=':',c='k',lw=0.3)

ax_sol.plot(t,x,'k',lw=0.5)
ax_sol.plot(t+params["tau"],x,'k',lw=0.3, linestyle=":")
ax_sol.set_xlim(0, 200)
ax_sol.axhline(0,linestyle=':',c='k',lw=0.3)
ax_sol.axhline(1,linestyle=':',c='k',lw=0.3)


# ax_sol.tick_params(labelbottom=False)


ax_le.plot(t, le, 'k', lw=0.5)
ax_le.tick_params(labelbottom=False)

le_mean = np.mean(le)
le_delta1 = np.max(le - le_mean);
le_delta2 = np.max(le_mean - le);
le_delta = min(le_delta1, le_delta2)/8;
ax_le.dataLim.y0 = le_mean - le_delta;
ax_le.dataLim.y1 = le_mean + le_delta;

window = len(le)//10
le_std = pd.Series(le).rolling(window=window).std()
ax_le_std.plot(t, le_std, 'k', lw=0.5)
ax_le_std.set_yscale("log")
le_final = np.mean(le[-window:])



ax_le.axhline(le_final,linestyle=':',c='k',lw=0.3)

ax_le.text(x=np.max(t), y=le_final+le_delta*0.1, s=f'$\\lambda(t_f) \\approx {le_final:.8f}$',va="bottom", ha="right")


ax_all = [ax_sol, ax_le, ax_le_std]

set_ticks(ax_all)

set_labels(ax_sol, "$t$", "$x(t)$")
set_labels(ax_le, None, "$\lambda(t)$")
set_labels(ax_le_std, "$t$", "$\sigma(t)$")

plt.savefig(f"output_img/{filename}.pdf", format="pdf", dpi=1000,bbox_inches='tight',pad_inches=0.01)



In [ ]:
params = dict(
    test = "test"
)

script = "test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"../output/bin/{filename}.bin"

my.run_cpp(script, params_str=params_str, recompile=True, flags="-O3")

# no matching function for call to ‘DDE<1, 1, 1, 1>::DDE
# (std::array<std::function<std::array<double, 1>(const std::array<double, 2>&)>, 2>, 
# std::array<std::function<std::array<double, 1>(const std::array<double, 2>&, const std::array<double, 2>&)>, 2>, 
# Vec<1>, 
# Func_b&, Func_db&, Vec<1>)’
  # 114 |         DDE<1,1,1,1>(array{f0,f1}, array{df0, df1}, Vec<1>{tau}, B, B_derivative, Vec<1>{1})

In [1027]:
params = dict(
    test = "test"
)

script = "test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"../output/bin/{filename}.bin"

my.run_cpp(script, params_str=params_str, recompile=True, flags="-g -O3")

# no matching function for call to ‘DDE<1, 1, 1, 1>::DDE
# (std::array<std::function<std::array<double, 1>(const std::array<double, 2>&)>, 2>, 
# std::array<std::function<std::array<double, 1>(const std::array<double, 2>&, const std::array<double, 2>&)>, 2>, 
# Vec<1>, 
# Func_b&, Func_db&, Vec<1>)’
  # 114 |         DDE<1,1,1,1>(array{f0,f1}, array{df0, df1}, Vec<1>{tau}, B, B_derivative, Vec<1>{1})

../cpp/test.cpp:23:19: error: expected ‘;’ after struct definition
   23 | struct test_arg {}
      |                   ^
      |                   ;
../cpp/test.cpp:28:21: error: ‘size’ is not a type
   28 | template <template <size t a, size_t b> typename test_arg>
      |                     ^~~~
../cpp/test.cpp:28:28: error: expected ‘>’ before ‘a’
   28 | template <template <size t a, size_t b> typename test_arg>
      |                            ^
../cpp/test.cpp:28:28: error: expected ‘class’ or ‘typename’ before ‘a’
../cpp/test.cpp:28:50: error: expected nested-name-specifier before ‘test_arg’
   28 | template <template <size t a, size_t b> typename test_arg>
      |                                                  ^~~~~~~~
../cpp/test.cpp:28:58: error: expected initializer before ‘>’ token
   28 | template <template <size t a, size_t b> typename test_arg>
      |                                                          ^
../cpp/test.cpp: In function ‘int main(int, char**)’:
.

ERROR: compiler exited and returned 256
